In [1]:

list_of_packages <- c("ggplot2", "dplyr", "DESeq2","biomaRt","tidyr", "tidyverse")
for(package in list_of_packages){
suppressPackageStartupMessages(suppressMessages(suppressWarnings(library(package,character.only=TRUE))))
}

In [2]:
figures_dir<-file.path("../figures")
if(!dir.exists(figures_dir)){
 dir.create(figures_dir, showWarnings = FALSE, recursive = TRUE)
}

In [3]:
dataset1 <- read.table("../data/U118mg/GSE152291_raw_counts_GRCh38.p13_NCBI.tsv", header=TRUE, sep="\t")
dataset2 <- read.table("../data/U118mg/GSE48865_raw_counts_GRCh38.p13_NCBI.tsv", header=TRUE, sep="\t")

# keep only GSM4610662 and GSM4610663 columns from dataset1
dataset1 <- dataset1 %>% dplyr::select(GeneID,GSM4610662, GSM4610663)
dim(dataset1)
dim(dataset2)
# combine the two datasets 
merged_dataset <- merge(dataset1, dataset2, by="GeneID", all=TRUE)
dim(merged_dataset)
head(merged_dataset)

[1] 39376     3

[1] 39376   275

[1] 39376   277

,GeneID,GSM4610662,GSM4610663,GSM1185864,GSM1185865,GSM1185866,GSM1185867,GSM1185868,GSM1185869,GSM1185870,⋯,GSM1186128,GSM1186129,GSM1186130,GSM1186131,GSM1186132,GSM1186133,GSM1186134,GSM1186135,GSM1186136,GSM1186137
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,515,543,124,88,20,77,34,63,100,⋯,164,94,103,139,130,45,145,26,71,103
2,2,4,3,7406,4694,9879,14567,6395,30121,5409,⋯,34234,10055,7369,9552,16688,7280,21142,10219,6204,10714
3,3,0,0,5,2,2,7,2,4,3,⋯,1,5,0,3,4,3,2,1,0,4
4,9,150,179,57,12,50,21,14,71,57,⋯,24,22,42,59,71,23,33,13,16,62
5,10,3,2,18,3,1,2,0,33,24,⋯,2,24,24,32,13,1,7,3,16,5
6,12,0,1,4403,248,773,167,29,13993,3840,⋯,20279,503,644,2300,1721,514,14871,8055,13021,1128


In [4]:
genes <- c(
    "GSDMA",
    "GSDMB",
    "GSDMC",
    "GSDMD",
    "GSDME",
    "TLR4",
    "TLR5",
    "CASP1",
    "CASP2",
    "CASP3",
    "CASP4",
    "CASP5",
    "CASP6",
    "CASP7",
    "CASP8",
    "CASP9",
    "CASP10",
    "CASP11"
)


In [5]:
mart <- useMart(biomart="ENSEMBL_MART_ENSEMBL", dataset="hsapiens_gene_ensembl")

# run and if if an error occurs, try again
while (TRUE) {
    tryCatch({
        entrezgene_id_list <- getBM(
            mart = mart,
            attributes = c('hgnc_symbol','entrezgene_id', "transcript_length"),
            filter = 'hgnc_symbol',
            values = genes,
            uniqueRows = TRUE)
        break
    }, error = function(e) {
        print(e)
    })
}
head(entrezgene_id_list)

,hgnc_symbol,entrezgene_id,transcript_length
,<chr>,<int>,<int>
1,CASP1,834,2505
2,CASP1,834,832
3,CASP1,834,424
4,CASP1,834,1061
5,CASP1,834,1521
6,CASP1,834,1307


In [6]:

# get the average transcript length for each gene group by hgnc_symbol and entrezgene_id
entrezgene_id_list <- entrezgene_id_list %>% 
    group_by(hgnc_symbol, entrezgene_id) %>%
    summarise(transcript_length = mean(transcript_length)) %>%
    ungroup()
    # get only genes that are present in the entrezgene_id_column of the entrezgene_id_list
merged_dataset <- merged_dataset[merged_dataset$GeneID %in% entrezgene_id_list$entrezgene_id,]

# convert entrezgene_id_list from a tibble to a data.frame
entrezgene_id_list <- as.data.frame(entrezgene_id_list)
merged_dataset <- as.data.frame(merged_dataset)
merged_dataset$entrezgene_id <- as.character(merged_dataset$GeneID)

`summarise()` has grouped output by 'hgnc_symbol'. You can override using the `.groups` argument.


In [7]:
head(merged_dataset,1)
head(entrezgene_id_list,1)

,GeneID,GSM4610662,GSM4610663,GSM1185864,GSM1185865,GSM1185866,GSM1185867,GSM1185868,GSM1185869,GSM1185870,⋯,GSM1186129,GSM1186130,GSM1186131,GSM1186132,GSM1186133,GSM1186134,GSM1186135,GSM1186136,GSM1186137,entrezgene_id
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>
598,834,397,517,246,69,44,161,70,493,116,⋯,122,68,175,375,100,469,187,129,147,834


,hgnc_symbol,entrezgene_id,transcript_length
,<chr>,<int>,<dbl>
1,CASP1,834,1429.045


In [8]:
merged_dataset$entrezgene_id
entrezgene_id_list$entrezgene_id

[1] "834"    "835"    "836"    "837"    "838"    "839"    "840"    "841"   
 [9] "842"    "843"    "1687"   "7099"   "7100"   "55876"  "56169"  "79792" 
[17] "284110"

[1]    834    843    835    836    837    838    839    840    841    842
[11] 284110  55876  56169  79792   1687   7099   7100

In [9]:
# merge the two datasets on the entrezgene_id column 
merged_dataset <- merge(merged_dataset, entrezgene_id_list, by="entrezgene_id")

In [10]:
# get the counts
# define the fpkm for each column that begins with GSM
for (i in 1:ncol(merged_dataset)) {
if (grepl("GSM", colnames(merged_dataset)[i])) {
 merged_dataset[,colnames(merged_dataset)[i]] <- merged_dataset[,colnames(merged_dataset)[i]] / merged_dataset$transcript_length
}
}


In [11]:
# remove GeneID column
merged_dataset <- merged_dataset[,!grepl("GeneID", colnames(merged_dataset))]
# convert to tibble

merged_dataset <- as_tibble(merged_dataset)
head(merged_dataset)

entrezgene_id,GSM4610662,GSM4610663,GSM1185864,GSM1185865,GSM1185866,GSM1185867,GSM1185868,GSM1185869,GSM1185870,⋯,GSM1186130,GSM1186131,GSM1186132,GSM1186133,GSM1186134,GSM1186135,GSM1186136,GSM1186137,hgnc_symbol,transcript_length
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
1687,0.6941307614,0.733485194,0.81830078,0.5183928658,0.5143217176,0.392865798,0.468182038,0.614743372,0.59845878,⋯,0.53196336,0.83458537,1.556535647,0.688024039,0.606601076,0.5367130325,0.263267581,0.726699947,GSDME,1473.786
284110,0.0007095553,0.002838221,0.02199622,0.0007095553,0.0007095553,0.003547777,0.002128666,0.002128666,0.02341533,⋯,0.02412488,0.05605487,0.007805109,0.001419111,0.007805109,0.0078051088,0.005676443,0.000000000,GSDMA,1409.333
55876,0.1463725074,0.160514779,0.05303352,0.0516192901,0.0905105360,0.052326404,0.056569085,0.069297129,0.06717579,⋯,0.07071136,0.14849385,0.063640221,0.057276199,0.039598359,0.1760712770,0.050912176,0.258096450,GSDMB,1414.200
56169,0.0046000511,0.010733453,0.05290059,0.0099667774,0.0007666752,0.003066701,0.000000000,0.045233836,0.06440072,⋯,0.05903399,0.07743419,0.075900843,0.001533350,0.015333504,0.0007666752,0.019933555,0.004600051,GSDMC,1304.333
7099,0.0308620604,0.028290222,0.18992037,0.0579652802,0.1946683812,0.229289282,0.050051931,0.284682724,0.45185222,⋯,0.13670310,0.05855878,0.577674465,0.226717444,0.248676987,0.1194915673,0.125426579,0.108215045,TLR4,5054.750
7100,0.0030670927,0.002726305,0.02385517,0.0129499468,0.0061341853,0.027263046,0.006815761,0.072247071,0.02589989,⋯,0.02726305,0.03169329,0.147902023,0.020788072,0.058956337,0.0719062833,0.024195953,0.016017039,TLR5,2934.375


In [12]:

# melt the dataset to long format
merged_dataset_long <- merged_dataset %>%
    dplyr::select(entrezgene_id, hgnc_symbol, transcript_length, starts_with("GSM")) %>%
    pivot_longer(cols = starts_with("GSM"), names_to = "sample", values_to = "fpkm")
# add a column to indicate u118mg bool for GSM4610662 and GSM4610663
merged_dataset_long$u118mg <- ifelse(grepl("GSM4610662|GSM4610663", merged_dataset_long$sample), TRUE, FALSE)
head(merged_dataset_long)

entrezgene_id,hgnc_symbol,transcript_length,sample,fpkm,u118mg
<chr>,<chr>,<dbl>,<chr>,<dbl>,<lgl>
1687,GSDME,1473.786,GSM4610662,0.6941308,TRUE
1687,GSDME,1473.786,GSM4610663,0.7334852,TRUE
1687,GSDME,1473.786,GSM1185864,0.8183008,FALSE
1687,GSDME,1473.786,GSM1185865,0.5183929,FALSE
1687,GSDME,1473.786,GSM1185866,0.5143217,FALSE
1687,GSDME,1473.786,GSM1185867,0.3928658,FALSE


In [13]:
# get the mean and standard deviation for each gene for each sample
merged_dataset_long <- merged_dataset_long %>%
    group_by(hgnc_symbol, entrezgene_id,sample,u118mg)
head(merged_dataset_long)

entrezgene_id,hgnc_symbol,transcript_length,sample,fpkm,u118mg
<chr>,<chr>,<dbl>,<chr>,<dbl>,<lgl>
1687,GSDME,1473.786,GSM4610662,0.6941308,TRUE
1687,GSDME,1473.786,GSM4610663,0.7334852,TRUE
1687,GSDME,1473.786,GSM1185864,0.8183008,FALSE
1687,GSDME,1473.786,GSM1185865,0.5183929,FALSE
1687,GSDME,1473.786,GSM1185866,0.5143217,FALSE
1687,GSDME,1473.786,GSM1185867,0.3928658,FALSE


In [14]:
# save the dataset to a file parquet file
arrow::write_parquet(merged_dataset_long,"../data/U118mg/merged_dataset_long.parquet")